In [1]:
print('Hello LangChain')

Hello LangChain


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [3]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="Langserve는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [4]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    # model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000002362CD977D0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002362D234140> root_client=<openai.OpenAI object at 0x000002362B96A480> root_async_client=<openai.AsyncOpenAI object at 0x000002362CE8DE80> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [5]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(type(response.content))
    print("응답:", response.content)
    
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}


### LCEL
- Prompt + LLM을 Chain으로 연결하기

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")\n    ')

In [7]:
# chain 연결 (LCEL) prompt + llm 연결
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
- Prompt + LLM + OutputParser을 Chain으로 연결하기

In [8]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [9]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 컴퓨터가 데이터를 통해 스스로 학습하고, 그 학습을 통해 특정 작업을 수행하는 능력을 키우는 과정입니다. 쉽게 말해, 인공지능이 마치 학생처럼 선생님(개발자)이나 교과서(데이터)로부터 정보를 얻어 문제를 해결하는 방법을 배우는 것이라고 할 수 있습니다.

### 기본 개념

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는 수많은 데이터가 필요합니다. 이 데이터는 문제의 유형에 따라 달라집니다. 예를 들어, 이미지 인식 모델을 개발하려면 수백만 장의 이미지가 필요하고, 언어 번역 모델을 개발하려면 수많은 문장과 번역본이 필요합니다.

2. **데이터 전처리**: 수집된 데이터는 모델에 사용하기 전에 전처리 과정을 거칩니다. 이 과정에서는 데이터의 오류를 수정하거나, 데이터를 일정한 형태로 변환하는 작업이 이루어집니다.

3. **모델**: 모델은 인공지능이 학습하는 과정의 기본 틀이 됩니다. 모델의 구조는 문제의 복잡도와 데이터의 특성에 따라 결정됩니다.

4. **학습**: 모델에 데이터를 입력하고, 모델이 그 데이터를 분석하여 스스로 규칙이나 패턴을 찾도록 하는 과정입니다. 이 과정에서는 모델의 파라미터(모델의 변수)가 조정됩니다.

5. **평가**: 학습이 완료된 후, 모델이 얼마나 잘 작동하는지 평가합니다. 이를 위해 별도의 테스트 데이터셋을 사용하여 모델의 성능을 측정합니다.

### 학습 방법

- **감독 학습(Supervised Learning)**: 모델에게 데이터와 그에 따른 정답을 알려주면서 학습시키는 방법입니다. 예를 들어, 고양이 사진과 아닌 사진을 보여주며 이것이 고양이인지 아닌지 알려주면서 학습합니다.

- **비감독 학습(Unsupervised Learning)**: 모델에게 데이터만 제공하고, 스스로 그 데이터 속에서 패턴이나 규칙을 찾도록 하는 학습 방법입니다. 예를 들어, 비슷한 이미지를 자동으로 분류

In [10]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe 같은 Product가 있어"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 제품과 도구를 제공하여 개발자들이 인공지능(AI) 모델을 보다 쉽게 구축하고 활용할 수 있도록 지원합니다. 주요 제품은 다음과 같습니다:

1. **LangSmith**: 랭스미스는 랭체인에서 제공하는 플랫폼으로, 개발자가 랭체인 기반의 애플리케이션을 개발, 테스트, 배포할 수 있도록 도와줍니다. 랭스미스를 통해 개발자는 랭체인의 다양한 기능과 모델을 쉽게 통합하고, 애플리케이션의 성능을 모니터링하며, 필요한 경우 디버깅을 수행할 수 있습니다.

2. **LangServe**: 랭서브는 랭체인에서 제공하는 또 다른 중요한 제품으로, 개발자가 대규모 언어 모델을 쉽고 빠르게 배포하고 관리할 수 있도록 설계되었습니다. 랭서브를 사용하면 개발자는 모델의 성능을 최적화하고, 모델을 다양한 환경에서 손쉽게 배포할 수 있습니다.

이러한 제품들은 개발자가 랭체인의 강력한 기능을 활용하여 혁신적인 AI 기반 애플리케이션을 구축할 수 있도록 지원합니다. 랭체인 및 이와 관련된 제품들은 지속적으로 업데이트되고 있으며, 새로운 기능과 도구들이 추가되고 있습니다.

LangChain의 제품들은 주로 개발자와 기업을 대상으로 하며, 이들이 AI 기술을 보다 쉽게 접목하고 활용할 수 있도록 돕습니다. 랭체인 및 관련 제품에 대한 더 자세한 정보는 공식 웹사이트나 관련 문서를 참조하는 것이 좋습니다.


### Runnable의 stream() 함수 호출

In [11]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    # 스트리밍 출력 
    print(answer)
    
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

<generator object RunnableSequence.stream at 0x000002362D20AB60>
인공지능 모델의 학습 원리를 설명드리겠습니다.

**인공지능 모델의 학습 원리**

인공지능 모델의 학습은 일종의 데이터 기반의 문제 해결 방법입니다. 모델이 주어진 데이터를 분석하고 학습하여, 새로운 데이터에 대해서도 정확한 예측이나 판단을 내릴 수 있도록 하는 것입니다.

**단계 1: 데이터 수집**

인공지능 모델을 학습시키기 위해서는 먼저 데이터를 수집해야 합니다. 이 데이터는 문제의 성격에 따라 달라지며, 이미지, 텍스트, 오디오 등 다양한 형태일 수 있습니다.

**단계 2: 데이터 전처리**

수집된 데이터는 학습에 적합한 형태로 변환되어야 합니다. 이 단계에서는 데이터의 품질을 높이고, 노이즈를 제거하며, 데이터를 정규화하는 등의 작업이 수행됩니다.

**단계 3: 모델 선택**

적합한 모델을 선택합니다. 모델의 종류로는 신경망, 결정 트리, 선형 회귀 등 다양한 것이 있으며, 문제의 성격과 데이터의 특성에 따라 적절한 모델을 선택해야 합니다.

**단계 4: 모델 학습**

모델에 데이터를 입력하고, 모델이 데이터를 분석하여 학습합니다. 이 단계에서는 모델이 데이터의 패턴을 인식하고, 예측이나 판단에 필요한 규칙을 스스로 발견하도록 합니다.

**단계 5: 손실 함수 정의**

모델의 성능을 평가하기 위해 손실 함수를 정의합니다. 손실 함수는 모델의 예측 결과와 실제 결과의 차이 정도를 측정합니다.

**단계 6: 최적화 알고리즘 적용**

모델의 가중치를 업데이트하여 손실 함수를 최소화하는 최적화 알고리즘을 적용합니다. 이를 통해 모델은 학습 데이터를 더 잘 예측할 수 있게 됩니다.

**단계 7: 모델 평가**

학습이 완료된 모델을 평가합니다. 평가 지표로는 정확도, 정밀도, 재현율, F1 스코어 등이 있으며, 이를 통해 모델의 성능을 측정합니다.

**단계 8: 모델 배포**

평가 결과 만족할 만한 성능을 보이면, 모

### Multi Chain

- 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
- 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 10문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화
chain1 = prompt1 | llm | StrOutputParser()

In [13]:

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}


### PromptTemplate 여러개 연결하기

In [14]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [ ]:

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [ ]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 언어 패턴과 구조를 이해하고, 이를 바탕으로 자연어 처리 작업을 수행하는 인공지능 모델입니다. 이 모델은 강화 학습 및 지도 학습을 사용하여 미세 조정되었으며 인터넷에서 수집한 대규모 텍스트 데이터로 학습되었습니다.
Gemma 모델은 주어진 문맥에 따라 두뇌 회로망 원리를 차용하여 만들어진 인공지능 모델입니다. 언어 모델에 transformer 구조를 사용하여 만들어졌으며, 사전학습된 언어 모델을 기반으로 미세 조정을 거쳐 다양한 자연어 처리 작업에 활용할 수 있습니다. Gemma는 텍스트 생성, 요약, 번역, 질의 응답 등의 작업에 활용할 수 있습니다.
llama-4 모델은 메타에서 개발한 대규모 언어 모델입니다. 이 모델은 방대한 텍스트 데이터를 기반으로 학습되며, 이를 통해 자연어 처리 능력을 습득합니다. 학습 과정에서 모델은 입력된 텍스트의 패턴과 구조를 파악하고, 이를 바탕으로 다음에 나타날 단어 또는 문장을 예측하도록 훈련됩니다. 이처럼 llama-4 모델은 다양한 텍스트 데이터를 학습함으로써 자연어 이해 및 생성 능력을 향상시킵니다.


### ChatPromptTemplate

- SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 한국어로 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

딥러닝은 인공신경망을 사용하여 데이터를 학습하고, 패턴을 발견하며, 분류, 예측, 생성 등의 작업을 수행하는 기계 학습의 한 분야입니다. 딥러닝은 데이터의 복잡한 패턴을 자동으로 학습할 수 있는 심층 신경망을 사용하여, 이미지, 음성, 텍스트 등의 다양한 형태의 데이터를 처리할 수 있습니다.

딥러닝은 전통적인 기계 학습 방법에 비해 다음과 같은 장점을 가지고 있습니다.

*   복잡한 패턴 학습: 딥러닝은 데이터의 복잡한 패턴을 자동으로 학습할 수 있습니다.
*   대용량 데이터 처리: 딥러닝은 대용량 데이터를 처리할 수 있습니다.
*   높은 정확도: 딛러닝은 높은 정확도로 데이터를 분류하고 예측할 수 있습니다.

딥러닝은 다양한 분야에서 활용되고 있습니다. 예를 들어, 이미지 인식, 음성 인식, 자연어 처리, 추천 시스템, 자율 주행 자동차 등이 있습니다.

딥러닝의 핵심 개념은 다음과 같습니다.

*   인공신경망: 딥러닝은 인공신경망을 사용하여 데이터를 학습합니다.
*   심층 신경망: 딥러닝은 심층 신경망을 사용하여 복잡한 패턴을 학습합니다.
*   활성화 함수: 딥러닝은 활성화 함수를 사용하여 신경망의 출력값을 결정합니다.
*   손실 함수: 딥러닝은 손실 함수를 사용하여 신경망의 성능을 평가합니다.
*   최적화 알고리즘: 딥러닝은 최적화 알고리즘을 사용하여 신경망의 가중치를 업데이트합니다.

딥러닝의 대표적인 알고리즘은 다음과 같습니다.

*   합성곱 신경망 (CNN): 이미지 처리에 많이 사용됩니다.
*   순환 신경망 (RNN): 시퀀스 데이터 처리에 많이 사용됩니다.
*   장단기 메모리 (LSTM): 시퀀스 데이터 처리에 많이 사용됩니다.
*   트랜스포머: 자연어 처리에 많이 사용됩니다.

딥러닝은 인공지능 분야에서 매우 중요한 기술이며, 다양한 분야에서 활용되고 있습니다.


### FewShotPromptTemplate

- 프롬프트에서 예시를 제공 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 가장 작은 행성, 태양과 가장 가깝습니다.
2. **금성**: 매우 뜨겁고 밝은 행성입니다.
3. **지구**: 생명체가 사는 유일한 행성입니다.
4. **화성**: 붉은 행성, 우주 탐사의 대상입니다.
5. **목성**: 태양계에서 가장 큰 행성입니다.
6. **토성**: 아름다운 고리를 가진 행성입니다.
7. **천왕성**: 자전축이 기울어진 행성입니다.
8. **해왕성**: 가장 먼 행성, 매우 차갑습니다.


### PartialPromptTemplate

- 프롬프트의 입력 값에 함수 호출 이나 외부 API를 호출한 동적인 값을 대입할 수 있음

In [ ]:
from datetime import datetime
# from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"
season = get_current_season("south")

# 프롬프트 템플릿 정의 (부분 변수 적용)
# prompt = PromptTemplate(
#     template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
#     input_variables=["phenomenon"],  # 사용자 입력 필요
#     partial_variables={"season": get_current_season("south")}  # 동적으로 계절 값 할당
# )

prompt = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 특정 계절의 현상 질의
# query = prompt.format(phenomenon="태풍 발생")  # '태풍 발생'은 여름과 관련됨
chain = (
    {"season" : lambda x: season}
    | prompt
    | model
    | StrOutputParser()
)
# result = model.invoke(query)
result = chain.invoke({})


# 결과 출력
# print(f"🔹 프롬프트: {query}")
# print(f"🔹 모델 응답: {result.content}")

print(f"🔹 프롬프트: {prompt}")
print(f"🔹 모델 응답: {season}에 발생하는 자연 현상 :\n {result}")

🔹 프롬프트: input_variables=['season'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['season'], input_types={}, partial_variables={}, template='{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. 각 현상에 대해 간단한 설명을 포함해주세요.'), additional_kwargs={})]
🔹 모델 응답: 겨울에 발생하는 자연 현상 :
 겨울에 주로 발생하는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1.  **오로라**: 오로라는 태양풍이 지구 자기장에 의해 극지방으로 유입되면서 대기 입자와 충돌하여 발생하는 현상입니다. 오로라는 북극과 남극에서 주로 발생하며, 녹색, 빨간색, 보라색 등 다양한 색상으로 나타납니다. 오로라는 밤하늘을 아름답게 수놓아 사람들에게 감동을 주는 자연의 경이로움 중 하나입니다.
2.  **빙하**: 빙하는 극지방이나 고산 지역에서 발생하는 현상으로, 눈이 쌓여 압축되면서 형성된 얼음 덩어리입니다. 빙하는 기후 변화에 민감하며, 지구 온난화로 인해 빙하의 크기가 줄어들고 있습니다. 빙하는 지구의 기후 조절에 중요한 역할을 하며, 빙하의 변화는 해수면 상승과 같은 지구적인 영향을 미칩니다.
3.  **성층권의 극소용돌이**: 성층권 극소용돌이는 성층권에서 발생하는 대규모 기상 현상으로, 극지방에서 발생하는 강력한 소용돌이입니다. 이 소용돌이는 지구의 자전과 태양 복사 에너지로 인해 발생하며, 기상 패턴에 큰 영향을 미칩니다. 성층권 극소용돌이는 기상 예측에 중요한 요소이며, 기상 이변을 예측하는 데 도움이 됩니다.


In [ ]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영

prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)
print(prompt)

input_variables=[] input_types={} partial_variables={'info': '1달러 = 1365.14원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.'


In [ ]:
# LLM 모델 설정 (GPT-4o-mini 사용)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

🔹 프롬프트: 현재 1달러 = 1365.14원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.
🔹 모델 응답: ## 한국경제에 미치는 영향

1.  **수출 증가**: 원화 약세는 한국의 수출을 촉진합니다. 원화 가치가 하락하면, 한국의 상품과 서비스는 해외 시장에서 더 저렴하게 느껴져 수출이 증가할 수 있습니다. 이는 한국 경제에 긍정적인 영향을 미칠 수 있습니다.
2.  **수입 비용 증가**: 반면에, 원화 약세는 수입 비용을 증가시킵니다. 원화 가치가 하락하면, 수입 상품과 서비스의 가격이 상승하여 국내 물가가 상승할 수 있습니다. 이는 가계의 구매력을 감소시키고, 인플레이션을 유발할 수 있습니다.
3.  **외국인 투자 유치**: 원화 약세는 외국인 투자 유치에 도움이 될 수 있습니다. 원화 가치가 하락하면, 외국 투자자들은 한국의 자산에 투자할 때 더 많은 이익을 얻을 수 있을 것으로 기대하여 투자를 늘릴 수 있습니다.

## 향후 환율 예상

*   **글로벌 경제 상황**: 미국의 통화 정책, 중국의 경제 성장률, 유럽의 경제 상황 등 글로벌 경제 상황은 환율에 큰 영향을 미칩니다.
*   **한국의 경제 지표**: 한국의 경제 성장률, 물가 상승률, 무역 수지 등 경제 지표는 환율에 영향을 미칩니다.
*   **원유 가격**: 원유 가격은 수입 비용에 큰 영향을 미치므로, 환율에도 영향을 미칠 수 있습니다.

환율은 다양한 요인에 의해 변동하므로, 정확한 예측은 어렵습니다. 하지만, 현재의 경제 상황과 전문가들의 분석을 바탕으로, 향후 환율의 예상값을 추정할 수 있습니다.

## 2024년 7월 1일 - 2024년 12월 31일: 1,350원 - 1,450원

*   **글로벌 경제 성장**: 글로벌 경제가 회복세를 보이면서, 원화 가치가 상승할 수 있습니다.
*   **한국의 경제 성장**: 한국의 경제가 안정적인 성장세를 보이면서, 원화 가치가 상승할 수 